In [32]:
import torch, torchvision
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
import torch.nn.functional as tf
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat


# NYU Dataset 
# -> image is 216x216, input vector size = 46656  

class NYU_DepthDataset(Dataset):
    def __init__(self, mat_file):
      mat = loadmat(mat_file)
      images = torch.from_numpy(mat['images']).permute(3, 2, 0, 1)
      depths = torch.from_numpy(mat['depths']).permute(2, 0, 1)

      images_max = images.amax((2, 3), keepdim=True) 
      images_min = images.amin((2, 3), keepdim=True) 
      images = (images - images_min) / (images_max - images_min)

      depths_max = depths.amax((1, 2), keepdim=True) 
      depths_min = depths.amin((1, 2), keepdim=True) 
      depths = (depths - depths_min) / (depths_max - depths_min)

      self.rgbd = torch.cat((images, depths.unsqueeze(1)), dim=1)

      self.std, self.mean = torch.std_mean(self.rgbd, (0, 2, 3))
      self.rgbd = transforms.functional.normalize(self.rgbd, self.mean, self.std)

    def __len__(self):
        return self.rgbd.shape[0]

    def __getitem__(self, idx):
      return self.rgbd[idx]

# Note : to be filled
batch_size = 50
# kwargs = {'num_workers': 1, 'pin_memory': True}
nyu = NYU_DepthDataset('drive/My Drive/Colab Notebooks/nyu.mat')
nyu_train = Subset(nyu, range(0, 1159))
nyu_test = Subset(nyu, range(1159, len(nyu)))
trainloader = DataLoader(dataset=nyu_train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=torch.cuda.is_available())
testloader = DataLoader(dataset=nyu_test, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=torch.cuda.is_available())

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# initialize random seeds; select gpu device if available
torch.manual_seed(1)
torch.cuda.manual_seed(1) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [45]:
# function to display the reconstructed images
# this function can be called during training/testing
def display_images(in_, out, n=1, label=None, count=False):
    for N in range(n):
        if in_ is not None:
            in_pic = in_.data.cpu().view(-1, 216, 216)
            plt.figure(figsize=(20, 4))
            plt.suptitle(label + ' – reconstructed images', color='w', fontsize=20)
            for i in range(4):
                plt.subplot(1,4,i+1)
                plt.imshow(in_pic[i+4*N])
                plt.axis('off')
        out_pic = out.data.cpu().view(-1, 216, 216)
        plt.figure(figsize=(20, 6))
        for i in range(4):  
            plt.subplot(1,4,i+1)
            plt.imshow(out_pic[i+4*N])
            plt.axis('off')
            if count: plt.title(str(4 * N + i), color='w')

In [46]:
# creating basic VAE model for image reconstruction; depth encoding - pending

d = 10 # data points can be varied accordingly
class VAE(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(nn.Linear(46656, d ** 2),nn.ReLU(),nn.Linear(d ** 2, d * 2))
        self.decoder = nn.Sequential(nn.Linear(d, d ** 2),nn.ReLU(),nn.Linear(d ** 2, 46656),nn.Sigmoid(),)
        
        # Note : classes for Encoder/Decoder can be added to encode depths using convolutions
        # class Encoder(nn.Module):
        # class Decoder(nn.Module):

    def reparam(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()  # compute standard dev using non-negative logvariance
            eps = std.data.new(std.size()).normal_() # to get normal distribution for backprop in training
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu_logvar = self.encoder(x.view(-1, 46656)).view(-1, 2, d)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]  # calculating the value of 'mu' and 'logvar' from encoder
        z = self.reparam(mu, logvar) # finding z by reparameterization
        return self.decoder(z), mu, logvar # return the decoder output

model = VAE().to(device)

# Setting the optimiser
optimizer = torch.optim.Adam(model.parameters(),lr=0.001) # learning rate can be modified accordingly

# define the loss function -> using binary_cross_entropy loss with KL divergence
def loss_function(x_hat, x, mu, logvar):  # where x = instance of training; x_hat, mu, logvar = model(x); same concept can be applied for testing phase
    BCE = nn.functional.binary_cross_entropy(x_hat, x.view(-1, 46656), reduction='sum')
    KLDiv = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))
    return BCE + KLDiv


In [ ]:
# Training and testing the VAE

epochs = 20 # set higher epoch values - around 300 to 500
codes = dict(mu1=list(), var1=list()) #, y1=list())
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        for x in trainloader:
            x = x.to(device)
            x_hat, mu, logvar = model(x)
            loss = loss_function(x_hat, x, mu, logvar)
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
       
        print(f'====> Epoch: {epoch} Train loss: {train_loss / len(train_loader.dataset):.4f}')
    
    # Testing
    means, logvars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x in testloader:
            x = x.to(device)
            x_hat, mu, logvar = model(x)
            test_loss += loss_function(x_hat, x, mu, logvar).item()
            means.append(mu.detach())
            logvars.append(logvar.detach())
            #labels.append(y.detach())
     
    codes['mu1'].append(torch.cat(means))
    codes['var1'].append(torch.cat(logvars))
    #codes['y1'].append(torch.cat(labels))
    test_loss /= len(testloader.dataset)
    print(f'====> Test set loss: {test_loss:.4f}')
    display_images(x, x_hat, 1, f'Epoch {epoch}')

====> Test set loss: 129782.2379
====> Epoch: 1 Train loss: 103134393717360069364059996160.0000
====> Test set loss: 12108915273450506944512.0000
====> Epoch: 2 Train loss: 2354515323973104828416.0000
====> Test set loss: 239849410192720.2812
====> Epoch: 3 Train loss: 17400703865420.2793
====> Test set loss: 982075245805.1310
====> Epoch: 4 Train loss: 17235010404361.0625
====> Test set loss: 588698248120.4414
====> Epoch: 5 Train loss: 17239553659099.8789
====> Test set loss: 561812316012.1655
====> Epoch: 6 Train loss: 17239786650744.9102
====> Test set loss: 559479812633.4896
====> Epoch: 7 Train loss: 17239785122846.0059
====> Test set loss: 559273615019.3103
====> Epoch: 8 Train loss: 17239785785010.1113
====> Test set loss: 559257106316.8000
====> Epoch: 9 Train loss: 17239784787903.8750
====> Test set loss: 559256961844.5793
====> Epoch: 10 Train loss: 17239783680226.0566
====> Test set loss: 559256928170.5931


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


====> Epoch: 11 Train loss: 17239784602126.9785
====> Test set loss: 559256932064.1379
====> Epoch: 12 Train loss: 17239784414461.6191
====> Test set loss: 559256716472.0552
====> Epoch: 13 Train loss: 17239784445590.3223
====> Test set loss: 559256705551.5586
====> Epoch: 14 Train loss: 17239719598707.4375
====> Test set loss: 559256716108.2483
====> Epoch: 15 Train loss: 17239652900719.9180
====> Test set loss: 559256683935.0620
====> Epoch: 16 Train loss: 17239653098810.3398
====> Test set loss: 559256621424.0552
====> Epoch: 17 Train loss: 17239652851888.5000
====> Test set loss: 559256647582.5104
====> Epoch: 18 Train loss: 17239652752631.5195
====> Test set loss: 559256759535.4482
====> Epoch: 19 Train loss: 17239653023185.9941
====> Test set loss: 559256673000.2207
